<a href="https://colab.research.google.com/github/Waseem-687/Agentic-AI/blob/main/LangChain_Rag_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 4.5 MB/s eta 0:00:00


In [16]:
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
pinecone_api_key = userdata.get('PINECONE_API_KEY')
pc = Pinecone(api_key=pinecone_api_key)

In [17]:
import time

index_name = "langchan-rag-project"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [18]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("white, flower!")
vector[:3]

[0.05607491731643677, -0.0688314214348793, -0.06224929541349411]

In [19]:
vector = embeddings.embed_query("who was the shakespeare?!")
vector[:3]

[0.008599186316132545, -0.05972126126289368, -0.014227797277271748]

In [20]:
vector = embeddings.embed_query("who was the shakespeare?!")
vector[:3]

[0.008599186316132545, -0.05972126126289368, -0.014227797277271748]

In [21]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [22]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['7b42e829-34a4-4f9f-8f8b-b2c50a3a255f',
 '1e08b111-c2f4-464b-ac48-9df4087255e6',
 '2a79a465-94f2-4e24-949c-629d4722b8d1',
 'd71803be-080f-4f6d-9b65-57b9bd0d45d5',
 '488a355b-4ab8-4166-b3ed-5cdc467a1088',
 'eb35dba7-2ef0-46ea-99f4-29d581fb64cd',
 '5038afb9-5791-41ae-9065-3d2bed45508c',
 'dc6cad2d-ed68-4933-b525-d08984b5b278',
 'c0f360a4-5399-4f7b-888e-77b52e2e2e99',
 'a830e6aa-cb93-4d06-b687-6e3fb8e47e05']

In [23]:
len(documents)

10

In [24]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['027f532d-3341-4146-9fbf-6fb9d232b9a1',
 '345e5139-c9c6-4fe7-94ee-a28df0505428',
 'd671265c-1edb-4e98-9f9b-3aa55e6311a2',
 'b20a4a01-1bdb-48eb-8c98-06666f62a361',
 '580b9331-91ac-4d2a-bf04-6b3b25d8be60',
 '8e6cc72f-c1b6-4167-b57c-c1d67a86ebdb',
 '4316594c-11bc-47a1-98ec-fbdad8cc200a',
 '1ca9c4c2-f320-4d04-a51d-073193c8f13c',
 '192874f2-e494-4d70-a20c-259a328a49a4',
 '27d4acce-0be9-4ee2-b7f9-0a3dad8b62c0']

In [25]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=6,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


In [26]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=7, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [27]:
!pip install langchain-google-genai -qU
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="gemini-1.5-flash", temperature = 0,max_tokens=None,timeout= None, max_retries=2,)

In [37]:
def answer_to_user_query(query: str) -> str:

    results = vector_store.similarity_search(query, k=5)
    print(len(results))
    final_answer = llm.invoke(f"answer the user query: {query}, here are some reference to answer {results}")
    return final_answer

In [40]:
answer = answer_to_user_query("what is langchain?")

5


In [44]:
print(answer) # Print the answer string directly

Based on the provided text, LangChain is a framework used for building projects.  The tweets all mention using LangChain for new projects, but don't offer a definition of what LangChain *is*.  More information is needed to give a complete answer.

